In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 24 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

04 April 2023 ; 01:25:09


In [3]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,33,0,9.7,1.5,3.5,0.440,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,1,1
1,Bobby Portis,PF,27,MIL,67,20,25.9,5.6,11.3,0.499,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,1
2,Brook Lopez,C,34,MIL,76,76,30.3,6.0,11.4,0.528,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,0
3,Giannis Antetokounmpo,PF,28,MIL,62,62,32.1,11.3,20.4,0.553,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,0
4,Goran Dragić,PG,36,MIL,55,0,14.7,2.4,5.6,0.421,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Nikola Jokić,DEN,4.5,1.2,0.7,110,4.405970,1,0.170991
1,Jimmy Butler,MIA,1.9,1.8,0.3,112,3.703226,7,0.136805
2,Giannis Antetokounmpo,MIL,2.7,0.8,0.8,107,4.893548,1,0.124178
3,Jaren Jackson Jr.,MEM,2.2,1.1,3.0,105,4.920000,2,0.111609
4,Anthony Davis,LAL,1.7,1.0,2.0,108,4.888462,7,0.096416


In [7]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)